In [6]:
import pandas as pd
import pickle

with open('p5r_data.pkl', 'rb') as file:
    data = pickle.load(file)

df_persona = pd.read_csv('p5r_人格面具.csv', index_col=0)
df_compose = pd.read_csv('p5r_合成总表.csv')
df_item = pd.read_csv('p5r_道具图鉴.csv', index_col=0)

persona_level = dict(zip(df_persona['中文名'].tolist(), df_persona['等级'].tolist()))
df_compose['Level'] = df_compose['Result'].map(persona_level)
df_compose['Lv.1'] = df_compose['Name.1'].map(persona_level).fillna(df_compose['Lv.1']).fillna(-1).astype(int)

df_item['备注'] = df_item['备注'].str.split('/').str[-1]
df_item['Lv'] = df_item['备注'].map(persona_level).fillna(-1).astype(int)

df_compose.head()

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
0,6561.0,愚者,8,背负怪,愚者,11,俄耳普斯F,亚森,1
1,7077.0,愚者,8,背负怪,愚者,13,俄耳普斯·贼神F,亚森,1
2,6084.0,魔术师,5,猫妖精,魔术师,11,杰克霜精,皮克希,2
3,6084.0,魔术师,5,猫妖精,魔术师,11,杰克霜精,皮克希,2
4,6741.0,恋人,6,幸魂,恋人,13,天钿女命,皮克希,2


In [12]:
# 查询反向合成表
df_compose.loc[df_compose['Result'].str.contains('默基瑟德')]

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
13182,37818.0,命运之轮,42,阿里亚德涅·贼神,教皇,40,大僧正,默基瑟德,58
13183,37872.0,命运之轮,39,阿特罗波斯,魔术师,43,妖精女王,默基瑟德,58
13184,38625.0,女皇,40,诃梨帝母,魔术师,43,妖精女王,默基瑟德,58
13185,38952.0,月,33,天蛾人,愚者,49,巴古斯,默基瑟德,58
13186,39345.0,命运之轮,35,拉克西斯,魔术师,48,兰达,默基瑟德,58
...,...,...,...,...,...,...,...,...,...
13492,92289.0,月,16,辉夜,愚者,95,撒坦耶尔,默基瑟德,58
13493,92289.0,太阳,16,朱雀,愚者,95,撒坦耶尔,默基瑟德,58
13494,92985.0,月,18,黑软泥怪,愚者,95,撒坦耶尔,默基瑟德,58
13495,95976.0,正义,41,能天使,正义,89,梅塔特隆,默基瑟德,58


In [7]:
# 查询技能，可尝试多个idx
name = '杰克霜精'
print(f'可选表：{list(range(len(data[name])))}')
data[name][2].iloc[4:, 1:6]

可选表：[0, 1, 2, 3, 4]


,1,2,3,4,5
4,技能,消耗,解锁等级,解锁等级,描述
5,布芙,4 SP,自带,自带,对1名敌人造成冰冻属性的小量伤害，罕见使敌人陷入冻结
6,消除冰冻防御,6 SP,自带,自带,消除所有敌人的冰冻耐性，无法消除防冰之壁
7,拜斯堤,4 SP,自带,自带,治疗1名伙伴的燃烧、冻结、触电状态
8,玛哈布芙,10 SP,12,12,对所有敌人造成冰冻属性的小量伤害，罕见使敌人陷入冻结
9,拉坤达,8 SP,13,13,在3回合间，使1名敌人的防御力下降
10,冻结几率UP,NaN,15,15,NaN


In [10]:
# 查询任意种族，等级面具
df_persona.loc[df_persona['中文名'].str.contains('八')]
# df_persona.loc[df_persona['等级'] == 64]
# df_persona.loc[df_persona['塔罗牌'] == '审判']

,中文名,等级,英文名,日文名,塔罗牌,物,枪,火,冰,电,风,念,核,祝,咒
160,八咫乌,57,Yatagarasu,ヤタガラス,顾问官,-,-,-,-,-,耐,-,-,无,弱
176,八岐大蛇,64,Yamata-no-Orochi,ヤマタノオロチ,审判,耐,-,-,吸,-,耐,-,-,-,耐


In [16]:
# 指定等级范围查询一步能合成的面具
name = '默基瑟德'
level = "1 - 100"

level_range = level.split(' - ')
name_cond = (df_compose['Name'].str.contains(name)) | (df_compose['Name.1'].str.contains(name))
level_cond = (df_compose['Level'] > int(level_range[0])) & (df_compose['Level'] < int(level_range[1]))
df_compose.loc[name_cond & level_cond]
# df_compose.loc[name_cond & level_cond, ['Result', 'Level']].drop_duplicates('Result')     # 仅显示合成结果及等级

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
3335,37917.0,正义,9,天使,正义,58,默基瑟德,权天使,29
3336,39162.0,正义,14,大天使,正义,58,默基瑟德,权天使,29
3338,53661.0,正义,41,能天使,正义,58,默基瑟德,权天使,29
4126,37245.0,正义,58,默基瑟德,战车,3,阿珈席翁,天蛾人,33
4277,36930.0,正义,58,默基瑟德,恋人,2,皮克希,阿努比斯,34
...,...,...,...,...,...,...,...,...,...
20759,111837.0,隐者,89,隐形鬼,正义,58,默基瑟德,斯尔特,83
20975,60477.0,正义,58,默基瑟德,愚者,49,巴古斯,哈斯塔,84
21096,80025.0,正义,58,默基瑟德,愚者,67,邪恶霜精,哈斯塔,84
21239,102297.0,正义,58,默基瑟德,愚者,83,毗湿奴,哈斯塔,84


In [10]:
from search_route import search_route

start = '默基瑟德'  # 设置合成起点
end = '爱丽丝'      # 设置合成终点
head_num = 10       # 设置显示个数（按合成成本升序）
max_depth = 3       # 设置搜索最大深度

search_route(start, end, max_depth=max_depth, head_num=head_num)

爱丽丝 特殊合成方式:
    种族  Lv  Name
1  恶魔  82   贝利亚
2  恶魔  74  奈比洛斯 

默基瑟德 + 莉莉丝 -> 奈比洛斯 = 71730.0 -特殊合成-> 爱丽丝
默基瑟德 + 拜亚基 -> 奈比洛斯 = 83850.0 -特殊合成-> 爱丽丝
默基瑟德 + 圣达芬 -> 奈比洛斯 = 90585.0 -特殊合成-> 爱丽丝
默基瑟德 + 希路奇 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 109548.0 -特殊合成-> 爱丽丝
默基瑟德 + 飞天 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 110523.0 -特殊合成-> 爱丽丝
默基瑟德 + 天钿女命 -> 八岐大蛇 + 八咫乌 -> 奈比洛斯 = 114162.0 -特殊合成-> 爱丽丝
默基瑟德 + 凯尔皮 -> 阿难陀 + 阿巴顿 -> 奈比洛斯 = 114531.0 -特殊合成-> 爱丽丝
默基瑟德 + 比利士 -> 百臂巨人 + 库夫林 -> 奈比洛斯 = 114831.0 -特殊合成-> 爱丽丝
默基瑟德 + 塔姆林 -> 佳塔由 + 大元帅明王 -> 奈比洛斯 = 114909.0 -特殊合成-> 爱丽丝
默基瑟德 + 塔姆林 -> 佳塔由 + 奥伯隆 -> 奈比洛斯 = 116130.0 -特殊合成-> 爱丽丝


In [59]:
# 查询道具图鉴

cond_app = df_item['适用'].str.contains('杏')
cond_type = df_item['类型'].str.contains('近战武器')
cond_all = True
df_item.loc[cond_type & cond_app]

,中文名,日文名,类型,属性,追加效果,获取,适用,备注,Lv
97,燃火鞭,バーンウィップ,近战武器,攻98 命97,低概率附加燃烧,商店售卖,高卷杏,6月21,-1
102,山羊皮鞭,山羊革のウィップ,近战武器,攻86 命99,NaN,商店售卖,高卷杏,5月18,-1
107,裂肤者,スキンリッパー,近战武器,攻68 命97,低机率附加恐惧,商店售卖,高卷杏,5月18,-1
111,分铜锁,分銅鎖,近战武器,攻58 命99,NaN,商店售卖,高卷杏,4月17,-1
115,炽红鞭,ヒートウィップ,近战武器,攻48 命97,低机率附加燃烧,商店售卖,高卷杏,4月17,-1
152,大深渊之鞭,NaN,近战武器,攻320 命90,高机率附加冰冻,警报电刑,高卷杏,邪恶霜精,67
160,缚狼锁,グラウプニル,近战武器,攻312 命99,NaN,商店售卖,高卷杏,1月13,-1
171,炎魔鞭,バルログウィップ,近战武器,攻292 命97,中机率附加燃烧,商店售卖,高卷杏,1月13,-1
179,冰雪女王鞭·改,NaN,近战武器,攻280 命90,在战斗开始时发动斯库卡加,警报电刑,高卷杏,丝卡蒂,53
180,深渊之鞭,NaN,近战武器,攻280 命90,高机率附加冰冻,警报电刑,高卷杏,邪恶霜精,67
